In [16]:
import sys
import csv
import os
import shutil
import pandas as pd
import numpy as np
import decimal
from pyneuroml import pynml
from pyneuroml.pynml import print_comment_v
from pyneuroml.lems import LEMSSimulation

import neuroml as nml
import neuroml.writers as writers
from neuroml.utils import validate_neuroml2
import random
random.seed(12345)



os.getcwd()

'C:\\Users\\roche\\Thesis code nml'

## Rather complex but official way to generate a network file from a nml morphology

It is faster to take an existing net file and then with a short python script change the Cell_id everywhere to create the other networks,
this allows you to easily loop over cells and networks and separate them into different folders.

Normally, this method generates a file that refers to `<include href="C51A_scaled_exp_resample_5.cell.nml"/>`, meaning cells and networks should share the same folder. This creates a big mess when you want many networks. 

Therefore the file generated here will refer to `<include href="../Cells/C51A_scaled_exp_resample_5.cell.nml"/>` , thus the cell file should **not** be in the same folder as the network file, rather it should be one directory up and placed in a cells folder.

Moreover, the created networks will be stored in a pynmlNetworks folder

In [17]:
path = os.getcwd()
cell_id = 'C51A'
Temp = 37
net_id = "network_of_%s"%cell_id    
net_doc = nml.NeuroMLDocument(id='net_'+str(cell_id))

#include cell file from ../Cells/
cellfile = nml.IncludeType(href='../Cells/'+cell_id+'_scaled_exp_resample_5.cell.nml')
net_doc.includes.append(cellfile)

#create network
net = nml.Network(id=net_id, type="networkWithTemperature", temperature=str(Temp)+"degC")
net_doc.networks.append(net)

#create population
pop = nml.Population(id='population_of_%s'%cell_id,component=cell_id,type='populationList',size='1')
net.populations.append(pop)

loc = nml.Location(x='0',y='0',z='0')
inst = nml.Instance(id='0',location=loc)
pop.instances.append(inst)

#include current clamp 
t_delay = 50
t_duration = 200
Amp = '0.15nA'

Iclamp = nml.PulseGenerator(id='iclamp0',delay=str(t_delay)+'ms',duration=str(t_duration)+'ms', amplitude=Amp)
net_doc.pulse_generators.append(Iclamp)

#add Iclamp to cell
input_list = nml.InputList(id='Iclamp', component=Iclamp.id, populations=pop.id)
input = nml.Input(id='0', target = "../%s/0/"%(pop.id)+str(cell_id), segmentId="0", destination="synapses")
input_list.input.append(input)
net.input_lists.append(input_list)


#write the network file
net_file_name = 'nml_'+cell_id+'.net.nml'
writers.NeuroMLWriter.write(net_doc,net_file_name)
shutil.move('C:\\Users\\roche\\Thesis code nml\\'+str(net_file_name), 'C:\\Users\\roche\\Thesis code nml\\pynmlNetworks')

'C:\\Users\\roche\\Thesis code nml\\pynmlNetworks\\nml_C51A.net.nml'

In [18]:
validate_neuroml2('./pynmlNetworks/nml_'+cell_id+'.net.nml')


print('\nCell=' +str(cell_id))  
print('network=' +str(net_id)) 
print(pop.id, inst)
print(Iclamp.id, 'delay='+str(t_delay), 'duration='+str(t_duration), 'Amplitude='+str(Amp))

Validating ./pynmlNetworks/nml_C51A.net.nml against C:\Users\roche\anaconda3\lib\site-packages\neuroml\nml/NeuroML_v2.1.xsd
It's valid!

Cell=C51A
network=network_of_C51A
population_of_C51A Instance 0 at location: (0, 0, 0)
iclamp0 delay=50 duration=200 Amplitude=0.15nA


## LEMS simulation file creation

LEMS holy grail: https://github.com/NeuroML/pyNeuroML/blob/87838a5d48db603025f35be667ba1c44439c7ee9/pyneuroml/lems/LEMSSimulation.py#L18

In [29]:
#redefine net and cell in case you don't run the above code segment
sim_id = 'Test'
length = 300
step = 0.000025

recorded_segment = 0
recorded_variable = ["../%s/0/"%(pop.id)+str(cell_id)+"/"+str(recorded_segment)+"/v"]


LEMS = LEMSSimulation(sim_id, length , step , target = 'net_'+str(cell_id))
LEMS.include_neuroml2_file('./pynmlNetworks/nml_'+cell_id+'.net.nml')
LEMS.set_report_file('SimStat.txt')                                   # not necessary but out of interest to see EDEN vs NEURON 

disp_id = 'display_0'
LEMS.create_display(disp_id, 'IDK FIRST DISPLAY TEST', '-100', '70' )
LEMS.add_line_to_display(disp_id, recorded_segment, recorded_variable)

traces = 'voltage traces'
LEMS.create_output_file(traces,"%s.v.dat"%sim_id)
LEMS.add_column_to_output_file(traces, recorded_segment, recorded_variable)

filename = 'LEMS_'+str(sim_id)+'_'+str(cell_id)+'.xml'
LEMS.save_to_file(file_name = filename )
shutil.move('C:\\Users\\roche\\Thesis code nml\\'+str(filename), 'C:\\Users\\roche\\Thesis code nml\\LEMSFILES')


pyNeuroML >>> Written LEMS Simulation Test to file: LEMS_Test_C51A.xml


'C:\\Users\\roche\\Thesis code nml\\LEMSFILES\\LEMS_Test_C51A.xml'

## Running LEMS file

In [35]:
LEMS_file = path+f'/LEMSFILES/{filename}'
out_dir = path+'/Simulation result'
print(f'You are running a simulation of {LEMS_file} and saving the results to {out_dir}\n')

results={}
results = pynml.run_lems_with_jneuroml_neuron(
        LEMS_file, verbose=False, nogui=True, load_saved_data=True, exec_in_dir=out_dir,
        max_memory='1000M')


You are running a simulation of C:\Users\roche\Thesis code nml/LEMSFILES/LEMS_Test_C51A.xml and saving the results to C:\Users\roche\Thesis code nml/Simulation result

pyNeuroML >>> *** Problem running command: 
pyNeuroML >>>        Command 'java -Xmx1000M  -jar  "C:\Users\roche\anaconda3\lib\site-packages\pyneuroml\lib\jNeuroML-0.10.0-jar-with-dependencies.jar"  "C:\Users\roche\Thesis code nml/LEMSFILES/LEMS_Test_C51A.xml"  -neuron -run -compile -nogui' returned non-zero exit status 1.
pyNeuroML >>>  jNeuroML >>  
pyNeuroML >>> Error: execute_command_in_dir returned with output: None


SystemExit: -1